In [1]:
# importing libararies
import numpy as np
import pandas as pd


In [47]:
df_train = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')
df_train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [48]:
print(df_train.shape)
print(df_test.shape)

(550068, 12)
(233599, 11)


In [49]:
# Separating target column form the rest of the data set 
target = df_train['Purchase']
df_train.drop(['Purchase'] , axis = 1 , inplace =True)

In [50]:
# combining train and test dataset for further feature extraction
# but first let use a flag column so that we can separate the train and test data after feature extraction
df_train['flag'] = 1
df_test['flag'] = -1
df = pd.concat([df_train , df_test] )
#df.drop(['Purchase'] , 1 , inplace = True)

In [51]:
df.shape

(783667, 12)

In [52]:
# let's replace NaN values of Categories column to 0
df.fillna(0 , inplace = True)

In [53]:
df.isnull().sum()

User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
flag                          0
dtype: int64

## Feature Extraction

In [54]:
# First let add all 3 column of the  product categories and normalise it
df['Categories'] = df['Product_Category_1'] + df['Product_Category_2'] + df['Product_Category_3'] 


It's seen that all the product from the data belongs to the **"Product_Category_1"** while some of the Product belongs to two categories and some of them belongs to the all three categories.
So let's make it two factor column for the **"Product_Category_2"**  and **"Product_Category_3"** in which " If the product belongs to the  Product_Category_2 the value of the column will be 1 other wise 0 similar in the second column"

In [55]:
df['is_Product_Category_2'] = 0
df['is_Product_Category_3'] = 0
df.loc[(df['Product_Category_2'] > 0.0) , 'is_Product_Category_2' ] = 1
df.loc[(df['Product_Category_3'] > 0.0) , 'is_Product_Category_3'] = 1

In [56]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,flag,Categories,is_Product_Category_2,is_Product_Category_3
0,1000001,P00069042,F,0-17,10,A,2,0,3,0.0,0.0,1,3.0,0,0
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,1,21.0,1,1
2,1000001,P00087842,F,0-17,10,A,2,0,12,0.0,0.0,1,12.0,0,0
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,0.0,1,26.0,1,0
4,1000002,P00285442,M,55+,16,C,4+,0,8,0.0,0.0,1,8.0,0,0


In [57]:
df['is_Product_Category_2'].value_counts()

1    537685
0    245982
Name: is_Product_Category_2, dtype: int64

Let's add a column for how many product a particular user purchased from the hsop, and another column for how many custome there are for a particular product

In [58]:
df.loc[(df['flag'] ==1) , 'How_Many_Product'] = df.loc[(df['flag'] == 1)].groupby('User_ID')['Product_ID'].transform('count')
df.loc[(df['flag'] ==-1) , 'How_Many_Product'] = df.loc[(df['flag'] == -1)].groupby('User_ID')['Product_ID'].transform('count')
df.loc[(df['flag'] ==1) , 'How_Many_Customer'] = df.loc[(df['flag'] == 1)].groupby('Product_ID')['User_ID'].transform('count')
df.loc[(df['flag'] ==-1) , 'How_Many_Customer'] = df.loc[(df['flag'] == -1)].groupby('Product_ID')['User_ID'].transform('count')
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,flag,Categories,is_Product_Category_2,is_Product_Category_3,How_Many_Product,How_Many_Customer
0,1000001,P00069042,F,0-17,10,A,2,0,3,0.0,0.0,1,3.0,0,0,35.0,227.0
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,1,21.0,1,1,35.0,581.0
2,1000001,P00087842,F,0-17,10,A,2,0,12,0.0,0.0,1,12.0,0,0,35.0,102.0
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,0.0,1,26.0,1,0,35.0,341.0
4,1000002,P00285442,M,55+,16,C,4+,0,8,0.0,0.0,1,8.0,0,0,77.0,203.0


In [59]:
#add column "Age" and "Occupation" and column "CIty_Category" and 'Stay_In_Current_City_Years'
df['Age_Occupation'] = df['Age']+'_'+df['Occupation'].astype('str')
df['City_Cat_Years'] = df['City_Category']+'_'+df['Stay_In_Current_City_Years']
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,flag,Categories,is_Product_Category_2,is_Product_Category_3,How_Many_Product,How_Many_Customer,Age_Occupation,City_Cat_Years
0,1000001,P00069042,F,0-17,10,A,2,0,3,0.0,0.0,1,3.0,0,0,35.0,227.0,0-17_10,A_2
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,1,21.0,1,1,35.0,581.0,0-17_10,A_2
2,1000001,P00087842,F,0-17,10,A,2,0,12,0.0,0.0,1,12.0,0,0,35.0,102.0,0-17_10,A_2
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,0.0,1,26.0,1,0,35.0,341.0,0-17_10,A_2
4,1000002,P00285442,M,55+,16,C,4+,0,8,0.0,0.0,1,8.0,0,0,77.0,203.0,55+_16,C_4+


Let's drop some of the column that might not help in predicting the Purchase values

In [60]:
drop_col = ['Stay_In_Current_City_Years' , 'User_ID','Product_ID','Age','Occupation','City_Category','Product_Category_1',
           'Product_Category_2','Product_Category_3']

df.drop(drop_col , 1 , inplace=True)
df.head()

,Gender,Marital_Status,flag,Categories,is_Product_Category_2,is_Product_Category_3,How_Many_Product,How_Many_Customer,Age_Occupation,City_Cat_Years
0,F,0,1,3.0,0,0,35.0,227.0,0-17_10,A_2
1,F,0,1,21.0,1,1,35.0,581.0,0-17_10,A_2
2,F,0,1,12.0,0,0,35.0,102.0,0-17_10,A_2
3,F,0,1,26.0,1,0,35.0,341.0,0-17_10,A_2
4,M,0,1,8.0,0,0,77.0,203.0,55+_16,C_4+


In [61]:
from sklearn.preprocessing import OrdinalEncoder , OneHotEncoder, StandardScaler, normalize
from sklearn.model_selection import train_test_split , KFold



In [62]:
ord_enc = OrdinalEncoder()

cat_feats = ['Gender','Age_Occupation' ,'City_Cat_Years']

for col in cat_feats:
    x = df[col].values
    x = x.reshape(-1,1)
    df[col] = ord_enc.fit_transform(x)

In [63]:
df.head()

,Gender,Marital_Status,flag,Categories,is_Product_Category_2,is_Product_Category_3,How_Many_Product,How_Many_Customer,Age_Occupation,City_Cat_Years
0,0.0,0,1,3.0,0,0,35.0,227.0,2.0,2.0
1,0.0,0,1,21.0,1,1,35.0,581.0,2.0,2.0
2,0.0,0,1,12.0,0,0,35.0,102.0,2.0,2.0
3,0.0,0,1,26.0,1,0,35.0,341.0,2.0,2.0
4,1.0,0,1,8.0,0,0,77.0,203.0,122.0,14.0


In [64]:
std_scaler = StandardScaler()

num_feats = ['Categories' , 'How_Many_Product' ,'How_Many_Customer', ]

for col in num_feats:
    x = df[col].values
    x = x.reshape(-1,1)
    df[col] = std_scaler.fit_transform(x)

In [65]:
df.head()

,Gender,Marital_Status,flag,Categories,is_Product_Category_2,is_Product_Category_3,How_Many_Product,How_Many_Customer,Age_Occupation,City_Cat_Years
0,0.0,0,1,-1.464352,0,0,-0.890818,-0.433438,2.0,2.0
1,0.0,0,1,0.568558,1,1,-0.890818,0.615401,2.0,2.0
2,0.0,0,1,-0.447897,0,0,-0.890818,-0.803791,2.0,2.0
3,0.0,0,1,1.133255,1,0,-0.890818,-0.095677,2.0,2.0
4,1.0,0,1,-0.899655,0,0,-0.632188,-0.504546,122.0,14.0


Now let's separate the train and test data from the dataset

In [66]:
train = df.loc[(df['flag'] == 1)]
test = df.loc[(df['flag'] == -1)]

In [67]:
print(train.shape )
print(test.shape)

(550068, 10)
(233599, 10)


In [74]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

x_train , x_val , y_train , y_val = train_test_split(train , target ,  test_size = 0.2 )

lin_reg = LinearRegression()
lin_reg.fit(x_train , y_train)

y_pred = lin_reg.predict(x_val)

np.sqrt(mean_squared_error(y_val , y_pred))

4537.97819049316

In [75]:
y_pred = lin_reg.predict(test)

In [76]:
sub = pd.read_csv('dataset/sample_submission_V9Inaty.csv')

In [78]:
sub['Purchase'] = y_pred

In [80]:
sub.to_csv('submission1.0.csv' , index = False)

In [82]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor()

sgd_reg.fit(x_train , y_train)
y_pred = sgd_reg.predict(x_val)
np.sqrt(mean_squared_error(y_val , y_pred))

4559.1948696508725

In [110]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth = 16 , max_features = 10)
tree.fit(x_train , y_train)
y_pred = tree.predict(x_val)
np.sqrt(mean_squared_error(y_val , y_pred))

3021.2113757493717

In [91]:
y_pred = tree.predict(test)

In [92]:
sub['Purchase'] = y_pred

In [93]:
sub.head()

,Purchase,User_ID,Product_ID
0,11294.837209,1000004,P00128942
1,7304.093869,1000009,P00113442
2,6114.888295,1000010,P00288442
3,5831.648402,1000010,P00145342
4,10219.983522,1000011,P00053842


In [94]:
sub.to_csv('submission2.0.csv' , index = False)

In [100]:
from sklearn.model_selection import cross_val_score

lin_scores = cross_val_score(lin_reg , train , target , scoring = 'neg_mean_squared_error' , cv=10)
lin_rmse_score = np.sqrt(-lin_scores)

In [101]:
def display_score(scores):
    print("Scores :", scores)
    print("Mean :" , scores.mean())
    print('Standard deviation:' ,scores.std())
    
display_score(lin_scores)

Scores : [-20015122.23960451 -20586753.59979907 -20521512.63231518
 -20255647.32942709 -20501682.52220955 -20490295.1509675
 -20374973.94552603 -20566304.02484488 -20633340.92302728
 -23974877.07167136]
Mean : -20792050.943939246
Standard deviation: 1075292.5801834702


In [102]:
lin_rmse_score

array([4473.82635331, 4537.26278717, 4530.06761896, 4500.62743731,
       4527.87836875, 4526.62072091, 4513.86463527, 4535.00871276,
       4542.39374372, 4896.41471606])

In [104]:
tree_scores = cross_val_score(tree , train , target , scoring = 'neg_mean_squared_error' , cv = 10)
print('tree_rmse_score :', np.sqrt(-tree_scores))
display_score(tree_scores)

tree_rmse_score : [3049.44776511 3082.1224785  3095.58934504 3056.76708139 3112.87720384
 3052.19741837 3080.93129135 3077.53732483 3059.82663666 3179.03777812]
Scores : [ -9299131.67210434  -9499478.97249834  -9582673.39314219
  -9343824.98989908  -9690004.48615925  -9315909.08067437
  -9492137.62204275  -9471235.98575088  -9362539.04639388
 -10106281.19473358]
Mean : -9516321.644339863
Standard deviation: 229404.1580466421


In [112]:
from sklearn.ensemble import RandomForestRegressor

rnd_reg = RandomForestRegressor(n_estimators= 20 , max_depth = 16 ,  max_features=10)
rnd_scores = cross_val_score(rnd_reg ,  train , target , scoring = 'neg_mean_squared_error' ,  cv =10)
print('rnd_rmse_score: ' , np.sqrt(-rnd_scores))
display_score(rnd_scores)

rnd_rmse_score:  [2866.20461892 2903.35000729 2891.40599318 2866.68437823 2899.43035624
 2862.06156891 2895.11091118 2893.51167814 2887.04912355 3036.83381305]
Scores : [-8215128.91749385 -8429441.26482569 -8360228.61741274 -8217879.32440648
 -8406696.39071395 -8191396.424219   -8381667.18805635 -8372409.83150913
 -8335052.64180149 -9222359.60810247]
Mean : -8413226.020854112
Standard deviation: 281637.2879326308


In [114]:
rnd_reg.fit(train ,  target)
y_pred = rnd_reg.predict(test)

In [115]:
sub['Purchase'] = y_pred

In [116]:
sub.to_csv('submission.3.0.csv' ,  index = False)

In [ ]:
from sklearn.svm import SVR

svr_poly_reg = SVR(kernel = 'poly' , degree = 2)
svr_poly_reg.fit(x_train , y_train)
y_pred = svr_poly_reg.predict(x_val)
np.sqrt(mean_squared_error(y_val , y_pred))